In [6]:
!pip install -U scikeras tensorflow scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tenso

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, f1_score, ConfusionMatrixDisplay
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
uploaded = files.upload()
data = pd.read_csv(list(uploaded.keys())[0])

In [5]:
label_map = {"Hujan": 0, "Cerah": 1, "Berawan": 2}
data["cuaca"] = data["cuaca"].map(label_map)

X = data[["suhu", "kelembaban", "angin"]].dropna()
y = data.loc[X.index, "cuaca"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def create_model(neurons=32, dropout_rate=0.2):
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_rate),
        Dense(neurons // 2, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=create_model,
    verbose=0,
    callbacks=[EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)]
)

param_grid = {
    'model__neurons': [16, 32],
    'model__dropout_rate': [0.0, 0.2],
    'batch_size': [16],
    'epochs': [20]
}

grid = GridSearchCV(model, param_grid, scoring='f1_macro', cv=2, verbose=1)
grid.fit(X_train, y_train)

print("\nBest Score:", grid.best_score_)
print("Best Params:", grid.best_params_)
y_pred = grid.best_estimator_.predict(X_test)
if y_pred.ndim > 1:
    y_pred = np.argmax(y_pred, axis=1)

print("\nAkurasi:", f"{accuracy_score(y_test, y_pred):.4f}")
print("F1 Macro:", f"{f1_score(y_test, y_pred, average='macro'):.4f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, target_names=list(label_map.keys())))

ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=list(label_map.keys()))
plt.title("Confusion Matrix - Neural Network")
plt.show()


Saving weather.csv to weather.csv


AttributeError: 'super' object has no attribute '__sklearn_tags__'